## Equivalence Checking

In [ ]:
# import packages
import allo
from allo.ir.types import float32

In [ ]:
# create gemm and schedule
def gemm(A: float32[32, 32], B: float32[32, 32]) -> float32[32, 32]:
  C: float32[32, 32] = 0
  for i, j in allo.grid(32, 32):
    for k in allo.reduction(32):
      C[i, j] += A[i, k] * B[k, j]
  return C

s1 = allo.customize(gemm)
s1.reorder("gemm:i", "gemm:j")
print(s1.module)

In [ ]:
s2 = allo.customize(gemm)
s2.buffer_at(s2.C, axis="i")
print(s2.module)

In [ ]:
# compare functional equivalence
verifier = allo.verify(s1, s2) # doesn't work?
assert verifier, "not equal"
print("equal")